# **Converting data in json format to pandas dataframe format**
SQuAD dataset is vary convoluted in json format, lets untangle the data and convert it to clean dataframe.
- I have written different functions for training and dev data as dev data have multiple answers for same questions unlike training data have only single..
- Try the code to understad the data better

In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np 
import pandas as pd 
import json 

from subprocess import check_output

In [20]:
def squad_json_to_dataframe_train(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.json_normalize(file , record_path )
    m = pd.json_normalize(file, record_path[:-1] )
    r = pd.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
    ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
    js['q_idx'] = ndx
    main = pd.concat([m[['id','question','context']].set_index('id'), js.set_index('q_idx')], axis=1, sort=False).reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

We have to define a different fuctions for Dev data because the the dev data is slightly different mainly there are multiple possible answer for a single question

In [21]:
def squad_json_to_dataframe_dev(input_file_path, record_path = ['data','paragraphs','qas','answers'],
                           verbose = 1):
    """
    input_file_path: path to the squad json file.
    record_path: path to deepest level in json file default value is
    ['data','paragraphs','qas','answers']
    verbose: 0 to suppress it default is 1
    """
    if verbose:
        print("Reading the json file")    
    file = json.loads(open(input_file_path).read())
    if verbose:
        print("processing...")
    # parsing different level's in the json file
    js = pd.json_normalize(file , record_path )
    m = pd.json_normalize(file, record_path[:-1] )
    r = pd.json_normalize(file,record_path[:-2])
    
    #combining it into single dataframe
    idx = np.repeat(r['context'].values, r.qas.str.len())
#     ndx  = np.repeat(m['id'].values,m['answers'].str.len())
    m['context'] = idx
#     js['q_idx'] = ndx
    main = m[['id','question','context','answers']].set_index('id').reset_index()
    main['c_id'] = main['context'].factorize()[0]
    if verbose:
        print("shape of the dataframe is {}".format(main.shape))
        print("Done")
    return main

In [22]:
# training data
input_file_path = 'train-v1.1.json'
record_path = ['data','paragraphs','qas','answers']
train = squad_json_to_dataframe_train(input_file_path=input_file_path,record_path=record_path)

Reading the json file
processing...
shape of the dataframe is (87599, 6)
Done


In [23]:
train.head()

,index,question,context,answer_start,text,c_id
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0
3,5733be284776f41900661181,What is the Grotto at Notre Dame?,"Architecturally, the school has a Catholic cha...",381,a Marian place of prayer and reflection,0
4,5733be284776f4190066117e,What sits on top of the Main Building at Notre...,"Architecturally, the school has a Catholic cha...",92,a golden statue of the Virgin Mary,0


In [25]:
# dev data
input_file_path = 'dev-v1.1.json'
record_path = ['data','paragraphs','qas','answers']
verbose = 0
dev = squad_json_to_dataframe_dev(input_file_path=input_file_path,record_path=record_path)

Reading the json file
processing...
shape of the dataframe is (10570, 5)
Done


In [26]:
dev.head()

,id,question,context,answers,c_id
0,56be4db0acb8001400a502ec,Which NFL team represented the AFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
1,56be4db0acb8001400a502ed,Which NFL team represented the NFC at Super Bo...,Super Bowl 50 was an American football game to...,"[{'answer_start': 249, 'text': 'Carolina Panth...",0
2,56be4db0acb8001400a502ee,Where did Super Bowl 50 take place?,Super Bowl 50 was an American football game to...,"[{'answer_start': 403, 'text': 'Santa Clara, C...",0
3,56be4db0acb8001400a502ef,Which NFL team won Super Bowl 50?,Super Bowl 50 was an American football game to...,"[{'answer_start': 177, 'text': 'Denver Broncos...",0
4,56be4db0acb8001400a502f0,What color was used to emphasize the 50th anni...,Super Bowl 50 was an American football game to...,"[{'answer_start': 488, 'text': 'gold'}, {'answ...",0


In [27]:
data = pd.DataFrame(dev)

In [28]:
print(data)

                             id  \
0      56be4db0acb8001400a502ec   
1      56be4db0acb8001400a502ed   
2      56be4db0acb8001400a502ee   
3      56be4db0acb8001400a502ef   
4      56be4db0acb8001400a502f0   
...                         ...   
10565  5737aafd1c456719005744fb   
10566  5737aafd1c456719005744fc   
10567  5737aafd1c456719005744fd   
10568  5737aafd1c456719005744fe   
10569  5737aafd1c456719005744ff   

                                                question  \
0      Which NFL team represented the AFC at Super Bo...   
1      Which NFL team represented the NFC at Super Bo...   
2                    Where did Super Bowl 50 take place?   
3                      Which NFL team won Super Bowl 50?   
4      What color was used to emphasize the 50th anni...   
...                                                  ...   
10565  What is the metric term less used than the New...   
10566  What is the kilogram-force sometimes reffered ...   
10567  What is a very seldom used unit 

In [29]:
train_data = pd.DataFrame(train)

In [30]:
print(train_data)

                          index  \
0      5733be284776f41900661182   
1      5733be284776f4190066117f   
2      5733be284776f41900661180   
3      5733be284776f41900661181   
4      5733be284776f4190066117e   
...                         ...   
87594  5735d259012e2f140011a09d   
87595  5735d259012e2f140011a09e   
87596  5735d259012e2f140011a09f   
87597  5735d259012e2f140011a0a0   
87598  5735d259012e2f140011a0a1   

                                                question  \
0      To whom did the Virgin Mary allegedly appear i...   
1      What is in front of the Notre Dame Main Building?   
2      The Basilica of the Sacred heart at Notre Dame...   
3                      What is the Grotto at Notre Dame?   
4      What sits on top of the Main Building at Notre...   
...                                                  ...   
87594  In what US state did Kathmandu first establish...   
87595               What was Yangon previously known as?   
87596  With what Belorussian city does 

In [31]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack

In [38]:
data['answers_text'] = data['answers'].apply(lambda x: x['text'])

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Apply TF-IDF separately to each column
tfidf_question = vectorizer.fit_transform(data['question'])
tfidf_context = vectorizer.fit_transform(data['context'])
tfidf_answers = vectorizer.fit_transform(data['answers_text'])

# Combine the TF-IDF matrices
tfidf_combined = hstack([tfidf_question, tfidf_context, tfidf_answers])

# Convert to array or keep it as a sparse matrix for efficient storage
tfidf_array = tfidf_combined.toarray()

# Display the result
print(tfidf_array)

TypeError: list indices must be integers or slices, not str